In [3]:
import cv2
import numpy as np

main_image = cv2.imread('cbn_grup2.jpg')

template = cv2.imread('cbn_grup.jpeg')
result = cv2.matchTemplate(main_image, template, cv2.TM_CCOEFF_NORMED)

# Find the location of the matched area
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

# Draw a rectangle around the matched area
h, w, _ = template.shape
cv2.rectangle(main_image, max_loc, (max_loc[0] + w, max_loc[1] + h), (0, 255, 0), 2)

# Display the result
cv2.imshow('Matching Result', cv2.resize(main_image,(800,800)))
cv2.imshow("Org Image",template)
cv2.waitKey(0)
cv2.destroyAllWindows()

Pyramid


In [1]:
import cv2
import numpy as np

image = cv2.imread('cbn.jpeg')

cv2.imshow('Original Image', image)

# Construct the Gaussian pyramid
gaussian_pyramid = [image]
for i in range(5):
    image = cv2.pyrDown(image)
    gaussian_pyramid.append(image)

for i, level_image in enumerate(gaussian_pyramid):
    cv2.imshow(f'Gaussian Pyramid Level {i}', level_image)


Choose any image and convert it into HSV and thn convert that HSV images into different sizes and save them on the disk with name as image480x480.jpg where the 480,will  be replaced with the size of the images

In [4]:
import cv2
# Read the original image
image = cv2.imread('mighty.jpg')
img2 = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
for i in range(5):
    image = cv2.pyrDown(image)
    cv2.imshow(f'Gaussian Pyramid Level {i}', image)
    cv2.imwrite(f'{image.shape[0]}x{image.shape[1]}.jpg', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

choose any image, apply any filter on that and then resize it where the size of all the generated images should be bigger than the original image and save it on the disk with same naming patterns as the goven question

In [5]:
import cv2
# Read the original image
image = cv2.imread('download.jpeg')
img2 = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
for i in range(5):
    image = cv2.pyrUp(image)
    cv2.imshow(f'Gaussian Pyramid Level {i}', image)
    cv2.imwrite(f'{image.shape[0]}x{image.shape[1]}.jpg', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

On a image check if the value of pixels are less than 169 replace those with Black and the others with white and obtain a final image. With the FInal image, add Blur to it with kernel size of 5x5 but make sure to choose the edge sensitive blurring.

In [3]:
import cv2
import numpy as np
image=cv2.imread("chick.jpg")
img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

lower_threshold = 169
_, binary_mask = cv2.threshold(img_hsv[:, :, 2], lower_threshold, 255, cv2.THRESH_BINARY)


cv2.imshow("Image",image)

cv2.waitKey(0)
cv2.destroyAllWindows()

finding circles in numbers

In [24]:
import cv2
import numpy as np


image = cv2.imread('num2.jpg', cv2.IMREAD_GRAYSCALE)
img=cv2.imread("num2.jpg")
blurred = cv2.GaussianBlur(image, (5, 5), 0)
cv2.imshow("Test",cv2.Canny(blurred,40,90))
circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1, minDist=50, param1=200, param2=30, minRadius=40, maxRadius=90)
if circles is not None:
    circles = np.uint16(np.around(circles))
    for circle in circles[0, :]:
        cv2.circle(image, (circle[0], circle[1]), circle[2], (0, 255, 0), 2)
cv2.imshow('Original image', img)
cv2.imshow('Hough Circles', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [35]:
import cv2
import numpy as np
image = cv2.imread('car.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 40, 100)
cv2.imshow("canny edges",edges)
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 150)
for line in lines:
    x1,y1,x2,y2 = line[0]
    cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 3)

cv2.imshow(str((lines)), image)
cv2.waitKey(0)
cv2.destroyAllWindows()

1)Preprocessing
2)Apply Thresholding
3)Morphological Transformations (DeNoising)
/ Removing Noise | Dilate/Erode

4)Creating a Marker Image
5)Apply Watershed Algo
6)Display Segemented
#Reason to Use Watershed 
#To get a segemented Image Output

In [42]:
import cv2
import numpy as np

image = cv2.imread('cbn_grup2.jpg')
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

_, thresh = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
# Perform morphological operations to clean the image
kernel = np.ones((3, 3), np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

# Identify the background area
sure_bg = cv2.dilate(opening, kernel, iterations=3)

# Perform distance transform
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)

# Threshold the distance transform to obtain sure foreground
_, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

# Subtract sure foreground from sure background to obtain unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

# Marker labeling for watershed algorithm
_, markers = cv2.connectedComponents(sure_fg)
markers = markers + 1
markers[unknown == 255] = 0

# Apply watershed algorithm
cv2.watershed(image, markers)
image[markers == -1] = [0, 0, 255]  # Mark watershed boundaries in red

cv2.imshow('Segmented Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [47]:
#GRABCUT MODEL
import cv2
import numpy as np

image = cv2.imread('cbn_grup2.jpg')
mask = np.zeros(image.shape[:2], np.uint8)
# Define the rectangle for initial GrabCut algorithm
rect = (50, 50, 450, 290)
# Initialize the foreground and background models
bgd_model = np.zeros((1, 65), np.float64)
fgd_model = np.zeros((1, 65), np.float64)
# Apply GrabCut algorithm
#GrabCut is Based on GraphCut Model along with Gaussian Mixture Model
cv2.grabCut(image, mask, rect, bgd_model, fgd_model, 5, cv2.GC_INIT_WITH_RECT)
# Modify the mask to create a binary mask for foreground and background
mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
# Multiply the original image with the binary mask to extract the foreground
result = image * mask2[:, :, np.newaxis]
cv2.imshow('Original Image', image)
cv2.imshow('Foreground Extraction', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

Harris Corner Detection
• Shi-Tomasi Corner Detector & Good Features
to Track
• Introduction to SIFT (Scale-Invariant Feature
Transform)
• Introduction to SURF (Speeded-Up Robust
Features)
• FAST Algorithm for Corner Detection
• BRIEF (Binary Robust Independent Elementary
Features)
• ORB (Oriented FAST and Rotated BRIEF)
• Feature Matching
16

In [2]:
import cv2
import numpy as np

image = cv2.imread('logo.png')

gaussian_pyramid = [image]
for i in range(6):
    image = cv2.pyrDown(image)
    gaussian_pyramid.append(image)

for i, level_image in enumerate(gaussian_pyramid):
    cv2.imshow(f'Gaussian Pyramid Level {i}', level_image)
# Construct the Laplacian pyramid from the Gaussian pyramid
laplacian_pyramid = [gaussian_pyramid[5]]
for i in range(5, 0, -1):
    expanded = cv2.pyrUp(gaussian_pyramid[i])
    laplacian = cv2.subtract(cv2.resize(gaussian_pyramid[i],(480,480)),cv2.resize(expanded,(480,480)))
    laplacian_pyramid.append(laplacian)

for i, level_image in enumerate(laplacian_pyramid):
    cv2.imshow(f'Laplacian Pyramid Level {i}', level_image)

cv2.waitKey(0)
cv2.destroyAllWindows()